# LLM Summarizer

### Autor: André Rizzo

Objetivo: Realizar o resumo de páginas web informadas pelo usuário.  

<br>

---

### Carregar bibliotecas

In [33]:
import os
import requests

from dotenv import load_dotenv
from bs4 import BeautifulSoup
from IPython.display import Markdown, display
from openai import OpenAI
from tqdm import tqdm

from markdown import markdown as md_to_html
from IPython.display import display, HTML

ModuleNotFoundError: No module named 'markdown'

### Carregar API key

In [3]:
load_dotenv(override=True)
api_key = os.getenv("OPENAI_API_KEY")
if not api_key:
    raise ValueError("Please set the OPENAI_API_KEY environment variable.")
else:
    print("API key loaded successfully.")

API key loaded successfully.


### Inicializa o cliente OpenAI para interagir com os serviços da OpenAI

In [4]:
openai = OpenAI()

#### Testar o funcionamento da API

In [7]:
message = "Oi ChatGPT ! Estou testando o seu funcionamento com o Jupyter Notebook. Você recebeu a minha mensagem ?"
response = openai.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": message}],
    temperature=0.7,
    max_tokens=150,
)
print(response.choices[0].message.content)

Oi! Sim, recebi a sua mensagem. Estou aqui para ajudar! Se você tiver alguma dúvida ou precisar de assistência com o Jupyter Notebook ou qualquer outro assunto, é só me avisar.


---

### Classe para extrair e limpar o conteúdo de uma página web a partir de sua URL

In [25]:
class Website:
    
    def __init__(self, url):
        """
        Extrai, limpa e analisa o conteúdo de um site.

        Args:
            url (str): URL do site a ser analisado.

        Returns:
            title: Título da página.
            content: Conteúdo da página.
        """
        self.url = url
        
        # Obter o conteúdo HTML da página
        response = requests.get(url)
        
        # Verificar se a requisição foi bem-sucedida
        if not response.status_code == 200:
            raise Exception(f"Failed to retrieve the webpage. Status code: {response.status_code}")
        
        # Criar o objeto BeautifulSoup para analisar o HTML
        soup = BeautifulSoup(response.content, 'html.parser')
        
        # Extrair o título da página
        self.title = soup.title.string if soup.title else "Título não encontrado"
        
        # Excluir tags que não possuem conteúdo relevante
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        
        # Extrair o conteúdo relevante da página
        self.content = soup.get_text(separator="\n", strip=True) 
        
        



#### Testar o scrapping

In [24]:
url = 'http://www.globo.com'

teste = Website(url)
print(teste.title)
print(teste.content)

globo.com - Absolutamente tudo sobre notícias, esportes e entretenimento
globo.com - Absolutamente tudo sobre notícias, esportes e entretenimento
Que tal ficar por dentro das notícias mais importantes em tempo real?
Ative as notificações da globo.com!
Agora não
Ativar
Menu
g1
o globo
valor
ge
cartola
globoplay
gshow
receitas
quem
podcast
g1
ge
gshow
globoplay
o globo
receitas
INSS diz que fará 'integral ressarcimento' de valor descontado ilegalmente
Diretora do INSS pede para que não haja corrida às agências
Como saber se você teve valor descontado indevidamente e o que fazer
Operação apreende Ferrari, relógios de luxo e obras de arte
Miriam Leitão: INSS foi alertado várias vezes
Lula e mais 18: veja lista de quem irá ao funeral do papa
SIGA: padre brasileiro fará meditação pré-conclave
Médico do papa: 'Ele não percebeu' os momentos finais
Caixão do papa receberá bolsa de moedas
Lula dá posse a Frederico Siqueira como novo ministro
Anistia: Motta decide adiar análise de urgência
Bolson

---

In [30]:
system_prompt = "Você é um assistente que analisa o conteúdo de um site \
    e fornece um resumo curto, ignorando textos que possam estar relacionados \
    à navegação. Responda em markdown."


def user_prompt_para_llm(website):
    """
    Cria um prompt (user prompt) para o modelo de linguagem.

    Args:
        website (Website): Objeto Website com o conteúdo a ser analisado.

    Returns:
        str: Prompt formatado.
    """
    user_prompt = f"\nEnviarei abaixo o conteúdo do site {website.title} e você \
            deverá criar um pequeno resumo das notícias publicads. O conteúdo deve ser \
            respondido em markdown."
    user_prompt += website.content
    return user_prompt
    



def mensagens_para_llm(website):
    """
    Cria uma lista de mensagens para o modelo de linguagem.

    Returns:
        list: Lista de mensagens formatadas.
    """
    mensagens = [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_para_llm(website)},
    ]
    return mensagens


def summarize(url):
    """
    Resumo do conteúdo de um site.

    Args:
        url (str): URL do site a ser analisado.

    Returns:
        str: Resumo do conteúdo da página.
    """
    # Criar o objeto Website
    website = Website(url)
    
    # Chama a API do OpenAI com o modelo de linguagem
    # passando os prompts formatados e o conteúdo do site
    response = openai.chat.completions.create(
        model="gpt-4o-mini",
        messages=mensagens_para_llm(website)
    )
    
    return response.choices[0].message.content


def resumo_website(url):
    """
    Exibe o texto formatado em Markdown no Jupyter Notebook.

    Args:
        text (str): Texto a ser exibido.
    """
    summary = summarize(url)
    display(Markdown(summary))

In [31]:
url = 'https://www.globo.com'
resumo_website(url)

```markdown
## Resumo das Notícias

- **INSS** anunciará o "integral ressarcimento" de valores descontados ilegalmente, recomendando que beneficiários evitem ir às agências.
- A **Operação** recente apreendeu uma Ferrari e itens de luxo, como relógios e obras de arte.
- **Lula** e outras 18 personalidades estarão presentes no funeral do Papa Francisco.
- **Bolsonaro** apresenta piora em seu quadro clínico, segundo boletim médico.
- Nos esportes, o Corinthians enfrenta dificuldades financeiras e o Santos não pagou rescisão de seu ex-treinador.
- O **Globo** planeja transmitir a primeira edição do novo Mundial de Clubes.
- **Entretenimento:** Xuxa relembra momentos dos 60 anos da Globo, e novos episódios de realities como "BBB" continuam a ser produzidos.

Para mais detalhes, visite o site.
```